In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_data(path):
    df = pd.read_csv(path)
    return df

In [ ]:
offloading_off = import_data('../../results/offloading/goracle/off/processed.csv')
offloading_only_udp = import_data('../../results/offloading/goracle/only-udp/processed.csv')
offloading_default = import_data('../../results/offloading/goracle/default/processed.csv')

In [ ]:
offloading_off['offloading'] = 'off'
offloading_only_udp['offloading'] = 'only-udp'
offloading_default['offloading'] = 'default'

data = pd.concat([
    offloading_off,
    offloading_only_udp,
    offloading_default,
], ignore_index=True)


In [ ]:
def plot_goodput(df):

    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    label = df.offloading.unique()
    implementation_order = {
        'lsquic': 1,
        'quiche': 2,
        'tcp-tls': 3
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    
    label_names = {
        'off': "Off",
        'default': "Default",
        'on': "All",
        'only-udp': "GSO/GRO"
    }
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'quiche': 'quiche',
        'quich': 'quiche',
        'tcp+tls': 'TCP/TLS',
    }

        
    colors = [ColorDarkerBlue, ColorGreen, ColorRed]
    patterns = [" "] * len(colors)
    linestyles = ['-'] * len(colors)
    markers = ['^', 'o', 's']
    
    def style(i):
        return {
            #"boxprops": dict(facecolor=c, color=c),
            "showmeans": True,
            "boxprops": dict(color=colors[i], linestyle=linestyles[i], facecolor="white"),
            "capprops": dict(color=colors[i], alpha=0),
            "whiskerprops": dict(alpha=0),
            "flierprops": dict(color=colors[i], markeredgecolor=colors[i]),
            #"medianprops": dict(color=c),
            #"meanprops": dict(color=c),
            "medianprops": dict(color=colors[i]),
            "meanprops": dict(markerfacecolor=colors[i], markeredgecolor=colors[i], marker=markers[i]),
            "whis": [5,95],
            "showfliers": False,
            "widths": .22,
            "positions": [x - 0.3 + 0.3 * i for x in range(len(label))],
            "patch_artist": True,
        }
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    plot_shades(label, ax1, 0.5)

    for idx, imp in enumerate(implementations):
        
        d = df.loc[(df.server == imp) & (df.client == imp)]
        bp1 = ax1.boxplot([d[d.offloading == c].goodput for c in label], labels=label,
            #hatch=patterns[0],
            **style(idx),
           )
    
    ax1.autoscale()
    
    ax1.set(xticks=range(len(label)), xticklabels=[label_names[c] for c in label])
    
    ax1.set_ylabel('Goodput [Gbit/s]')
    ax1.set_ylim(0, 9000)
    
    mkfunc = lambda x, pos: f'{(x * 1e-3):.0f}'
    mkformatter = matplotlib.ticker.FuncFormatter(mkfunc)
    ax1.yaxis.set_major_formatter(mkformatter)
    
    legend_elements = [
        # Patch(facecolor='white', edgecolor=colors[i], linestyle=linestyles[i], label=implementation_names[imp])
        mlines.Line2D([], [], color=colors[i], marker=markers[i], linestyle='None', label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    
    ax1.legend(handles=legend_elements, loc="upper left", ncol=1)
    fig.tight_layout()
    fig.savefig('offloading.pdf')


In [ ]:
plot_goodput(data)